# Research into Chatbot stuff + Langchain

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from openai_harmony import (
    load_harmony_encoding, HarmonyEncodingName,
    Conversation, Message, Role
)

print("Successful Imports...")

c:\Users\peter\OneDrive\Documents\GitHub\09-15-2025_discord-meeting-transcriptor\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successful Imports...


In [4]:
### This is code to RESET the vram
# Remove all loaded memory and models
import gc

# Delete pipeline, model, tokenizer, and encoder if they exist
if 'pipe' in globals():
    del pipe
if 'model' in globals():
    del model
if 'tok' in globals():
    del tok
if 'enc' in globals():
    del enc

# Force garbage collection
gc.collect()

# Clear CUDA cache
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

print("✓ VRAM cleared and memory freed!")

✓ VRAM cleared and memory freed!


In [ ]:
# Optional: Clear HuggingFace cache if model loading issues persist
# Uncomment and run if you need to re-download the model
"""
import shutil
from pathlib import Path

cache_dir = Path.home() / ".cache" / "huggingface" / "hub"
model_cache = cache_dir / "models--openai--gpt-oss-20b"

if model_cache.exists():
    shutil.rmtree(model_cache)
    print("✓ Model cache cleared! Re-run the loading cell to download fresh.")
else:
    print("No cache found for this model.")
"""
print("Cache clearing cell (currently disabled - uncomment to use)")

✓ Model cache cleared! Re-run the loading cell to download fresh.
Cache clearing cell (currently disabled - uncomment to use)


In [2]:
# Load GPT-OSS-20B model with Transformers pipeline
# Note: This model comes pre-quantized with Mxfp4, so it's already memory-efficient
from transformers import pipeline

model_id = "openai/gpt-oss-20b"

# Try to load with specific settings to handle the quantized model properly
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True,
    revision="main",  # Use main branch
)

# Load Harmony encoding for proper message parsing (optional for advanced use)
enc = load_harmony_encoding(HarmonyEncodingName.HARMONY_GPT_OSS)

print("✓ Model pipeline loaded (pre-quantized) and ready!")

`torch_dtype` is deprecated! Use `dtype` instead!
MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16
Loading checkpoint shards: 100%|██████████| 3/3 [00:20<00:00,  6.77s/it]
Some parameters are on the meta device because they were offloaded to the disk and cpu.
Device set to use cuda:0


✓ Model pipeline loaded (pre-quantized) and ready!


In [10]:
# Single-turn conversation function using pipeline
def ask_gpt_oss(user_message: str, system_prompt: str = "You are a helpful assistant.\nReasoning: medium") -> str:
    """
    Single-turn query to GPT-OSS-20B using Transformers pipeline.
    
    Args:
        user_message: The user's question
        system_prompt: System instructions (can include reasoning level)
    
    Returns:
        The assistant's final response text
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
    
    outputs = pipe(
        messages,
        max_new_tokens=256,
    )
    
    # Extract the assistant's response (last message in the generated conversation)
    return outputs[0]["generated_text"][-1]["content"]


# Test with RAG-related questions
print("Q1: Give two bullets on what RAG is.")
print(ask_gpt_oss("Give two bullets on what RAG is."))
print("\n" + "="*50 + "\n")

print("Q2: Using that, name one benefit of chunking transcripts.")
print(ask_gpt_oss("Using that, name one benefit of chunking transcripts."))
print("\n" + "="*50 + "\n")

print("Q3: Suggest one metric to evaluate retrieval quality.")
print(ask_gpt_oss("Suggest one metric to evaluate retrieval quality."))

Q1: Give two bullets on what RAG is.


KeyError: 'model.layers.11.mlp.experts.gate_up_proj'